In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import psutil
from collections import defaultdict
import datatable as dt
import lightgbm as lgb
from matplotlib import pyplot as plt
import riiideducation
import random
from sklearn.metrics import roc_auc_score
import gc
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

SAKT part learned from https://www.kaggle.com/mpware/sakt-fork 

In [ ]:
train=pd.read_csv('../input/riiid-test-answer-prediction/train.csv',nrows=1000000)
train.head()

In [ ]:
##calculate the recency of a student
train['lagtime'] = train.groupby('user_id')['timestamp'].shift()
train['lagtime']=train['timestamp']-train['lagtime']
train['lagtime'].fillna(0, inplace=True)
train.lagtime=train.lagtime.astype('int32')
#calculate mean time of prior questions
prior_time_agg = train.groupby('user_id')['prior_question_elapsed_time'].agg(['mean','sum'])
train['prior_time_mean_user'] = train['user_id'].map(prior_time_agg['mean'])
#train.prior_time_mean_user=train.prior_time_mean_user.astype('int64')

In [ ]:
#about prior question answered correctness
prior_answer_correctness=train.groupby('prior_question_had_explanation')['answered_correctly'].agg(['sum'])
train['prior_answer_correctness']=train['prior_question_had_explanation'].map(prior_answer_correctness['sum'])
train.prior_answer_correctness=train.prior_answer_correctness.astype('float16')

In [ ]:
correct_agg = train[train['answered_correctly']==1].groupby('user_id').answered_correctly.agg(['sum'])
train['correct_count_user'] = train['user_id'].map(correct_agg['sum'])
false_agg = train[train['answered_correctly']==0].groupby('user_id').answered_correctly.agg(['count'])
train['false_count_user'] = train['user_id'].map(false_agg['count'])
null_agg = train[train['answered_correctly']==-1].groupby('user_id').answered_correctly.agg(['count'])
train['null_count_user'] = train['user_id'].map(null_agg['count'])
#correctness
train['lag_correctness'] = train.groupby('user_id')["answered_correctly"].shift()
cum = train.groupby('user_id')['lag_correctness'].agg(['cumsum', 'cumcount'])
train['user_correctness'] = cum['cumsum'] / cum['cumcount']

In [ ]:
#about expainled questions
explain_agg_user= train[train['prior_question_had_explanation']==True].groupby('user_id').prior_question_had_explanation.agg(['count'])
train['explained_count_user'] = train['user_id'].map(explain_agg_user['count'])

In [ ]:
##grouped by user
#about time
#train["correct_count_user"]=train.groupby(train["user_id"])["answered_correctly"].count(1)
#train["false_count_user"]=train["answered_correctly"].groupby(train["user_id"]).count(0)
#train["null_count_user"]=train["answered_correctly"].groupby(train["user_id"]).count(-1)
train['prior_time_sum_user'] = train['user_id'].map(prior_time_agg['sum'])
train.prior_time_sum_user=train.prior_time_sum_user.astype('int32')
#train["prior_mean_correct_user"]=train["prior_time_sum_user"]/train["correct_count_user"]
#train["prior_mean_false_user"]=train["prior_time_sum_user"]/train["false_count_user"]
#train["prior_mean_null_user"]=train["prior_time_sum_user"]/train["null_count_user"]v

In [ ]:
#about content
content_agg = train.groupby('content_id')['answered_correctly'].agg(['sum', 'count','var'])
train['content_correctness'] = train['content_id'].map(content_agg['sum'] / content_agg['count'])
train.content_correctness=train.content_correctness.astype('float16')
train["attempt_no"] = 1
train.attempt_no=train.attempt_no.astype('int8')
train["attempt_no"] = train[["user_id","content_id",'attempt_no']].groupby(["user_id","content_id"])["attempt_no"].cumsum()
explain_agg_contain= train[train['prior_question_had_explanation']==True].groupby('content_id').prior_question_had_explanation.agg(['count'])
train['explained_count_content'] = train['content_id'].map(explain_agg_contain['count'])

In [ ]:
##lectures watched & lecture_vs_questions
lec_cum = train.groupby('user_id')['content_type_id'].agg(['cumsum', 'cumcount'])
train['user_lecture_cumsum'] = lec_cum['cumsum'] 
train['user_lecture_cumavg'] = lec_cum['cumsum'] / lec_cum['cumcount']
train.user_lecture_cumavg=train.user_lecture_cumavg.astype('float16')
train.user_lecture_cumsum=train.user_lecture_cumsum.astype('int8')
user_lecture_agg = train.groupby('user_id')['content_type_id'].agg(['sum', 'count'])
train['user_lecture_sum'] = train['user_id'].map(user_lecture_agg['sum'])
train['user_lecture_avg'] = train['user_id'].map(user_lecture_agg['sum'] / user_lecture_agg['count'])
train.user_lecture_avg=train.user_lecture_avg.astype('float16')
train.user_lecture_sum=train.user_lecture_sum.astype('int8')

In [ ]:
train.prior_question_had_explanation=train.prior_question_had_explanation.astype('bool')

In [ ]:
questions_df = pd.read_csv(
    '../input/riiid-test-answer-prediction/questions.csv', 
    usecols=[0,1,3,4],
    dtype={'question_id': 'int16','bundle_id': 'int16', 'part': 'int8','tags': 'str'},
    nrows=200000000
)

In [ ]:
### TSVD decomposition
from gensim import corpora,similarities,models
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')

questions_df['tags'].fillna("-1",inplace=True)

class getwords(object):
    def __iter__(self):
        for index,doc in enumerate(questions_df['tags']):
            if index%5000000==0:
                print(index)
            yield doc.split(' ')
corp = getwords()
dictionary=corpora.Dictionary(corp)
class doc2b(object):
    def __init__(self):
        print('MyCorpus')
    def __iter__(self):
        for index,doc in enumerate(questions_df['tags']):
            if index%5000000==0:
                print(index)
            yield dictionary.doc2bow(doc.split(' '))
corpus = doc2b()
tfidf_model = models.TfidfModel(corpus, id2word=dictionary)

tfidf_model.save("./tfidf.model")
dictionary.save("./dictionary.model")
corpus_tfidf = tfidf_model[corpus]
class tfidfv(object):
    def __init__(self):
        print('MyCorpus')
    def __iter__(self):
        for index,doc in enumerate(corpus_tfidf):
            if index%1000000==0:
                print(index)
            yield doc
corpus3 = tfidfv()
lsi_model = models.LsiModel(corpus3, id2word=dictionary,chunksize=2500000,num_topics=50)
lsi_model.save("./lsi_all_query.lsi")

def get_arg_max(single_list):
    max_index=0
    max_num=single_list[0][1]
    for index in range(len(single_list)-1):
        if max_num<single_list[index+1][1]:
            max_num=single_list[index+1][1]
            max_index=index+1
    return max_index
lsi_model=models.LsiModel.load("./lsi_all_query.lsi")
dictionary=corpora.Dictionary.load("./dictionary.model")
tfidf_model=corpora.Dictionary.load("./tfidf.model")
all_data_lsi=[]
for text in tqdm(questions_df['tags']):
    single_row_text=dictionary.doc2bow(text.split(' '))
    single_row_lsi_list=list(lsi_model[tfidf_model[single_row_text]])
    if len(single_row_lsi_list)==0:
        all_data_lsi.append(0)
        continue
    single_row_lsi=get_arg_max(single_row_lsi_list)
    all_data_lsi.append(single_row_lsi)
questions_df['tags_lsi']=all_data_lsi
questions_df.head(10)

In [ ]:
questions_df['part_bundle_id']=questions_df['part']*100000+questions_df['bundle_id']
questions_df.part_bundle_id=questions_df.part_bundle_id.astype('int32')
tag = questions_df['tags'].str.split(" ", n = 10, expand = True)
tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']
#

tag.fillna(0, inplace=True)
tag = tag.astype('int16')
questions_df =  pd.concat([questions_df,tag],axis=1).drop(['tags'],axis=1)
questions_df.rename(columns={'question_id':'content_id'}, inplace=True)
questions_df['content_correctness'] = questions_df['content_id'].map(content_agg['sum'] / content_agg['count'])
questions_df.content_correctness=questions_df.content_correctness.astype('float16')
#questions_df['content_correctness_std'] = questions_df['content_id'].map(content_agg['var'])
#questions_df.content_correctness_std=questions_df.content_correctness_std.astype('float16')
part_agg = questions_df.groupby('part')['content_correctness'].agg(['mean', 'var'])
questions_df['part_correctness_mean'] = questions_df['part'].map(part_agg['mean'])
#questions_df['part_correctness_std'] = questions_df['part'].map(part_agg['var'])
questions_df.part_correctness_mean=questions_df.part_correctness_mean.astype('float16')
#questions_df.part_correctness_std=questions_df.part_correctness_std.astype('float16')
bundle_agg = questions_df.groupby('bundle_id')['content_correctness'].agg(['mean'])
questions_df['bundle_correctness'] = questions_df['bundle_id'].map(bundle_agg['mean'])
questions_df.bundle_correctness=questions_df.bundle_correctness.astype('float16')
tags1_agg = questions_df.groupby('tags1')['content_correctness'].agg(['mean', 'var'])
questions_df['tags1_correctness_mean'] = questions_df['tags1'].map(tags1_agg['mean'])
tags2_agg = questions_df.groupby('tags2')['content_correctness'].agg(['mean', 'var'])
questions_df['tags2_correctness_mean'] = questions_df['tags2'].map(tags2_agg['mean'])
tags3_agg = questions_df.groupby('tags3')['content_correctness'].agg(['mean', 'var'])
questions_df['tags3_correctness_mean'] = questions_df['tags3'].map(tags3_agg['mean'])
tags_lsi_agg = questions_df.groupby('tags_lsi')['content_correctness'].agg(['mean', 'var'])
questions_df['tags_lsi_correctness_mean'] = questions_df['tags_lsi'].map(tags_lsi_agg['mean'])
questions_df.tags_lsi_correctness_mean=questions_df.tags_lsi_correctness_mean.astype('float16')
#questions_df['tags1_correctness_std'] = questions_df['tags1'].map(tags1_agg['var'])
questions_df.tags1_correctness_mean=questions_df.tags1_correctness_mean.astype('float16')
#questions_df.tags1_correctness_std=questions_df.tags1_correctness_std.astype('float16')
questions_df.drop(columns=['content_correctness'], inplace=True)

In [ ]:
#train_question=train[train["content_type_id"] == 0].merge(questions_df,left_on='content_id', how="left")
train_question = pd.merge(train[train["content_type_id"] == 0], questions_df, on='content_id', how='left',right_index=True)
train_question.prior_question_had_explanation=train_question.prior_question_had_explanation.astype('bool')

In [ ]:
train_question.info()

In [ ]:
## use lgbm to select important features
features = ['lagtime',
    #'lagtime_mean',
   'content_id',
   # 'task_container_id',
    'user_lecture_cumsum', # X
    'user_lecture_cumavg',
    'prior_question_elapsed_time',
    #'user_correctness',
    #'lag_correctness',
    #'correct_count_user',
    #'false_count_user',
    #'null_count_user',
    'prior_time_sum_user',
    'prior_answer_correctness',
    #'prior_mean_correct_user',
    #'prior_mean_false_user',
    #'prior_mean_null_user',
    #'user_correct_cumcount', 
    #'user_correct_cumsum', 
    'content_correctness',
    'user_lecture_cumsum',
    'user_lecture_cumavg',
    'user_lecture_sum',
    'user_lecture_avg',
   # 'content_correctness_std',
    #'content_count',
    #'content_sum', 
    #'task_container_correctness',
   # 'task_container_std',
   # 'task_container_sum',
    #'bundle_correctness',
    'attempt_no',
    'part',
    'part_correctness_mean',
   # 'part_correctness_std',
'tags1',
'tags_lsi_correctness_mean',
'tags2',
'tags3',
'tags_lsi',
    'bundle_id',
  #  'part_bundle_id',
    #'explained_count_user', 
    'explained_count_content',
    'prior_question_had_explanation',
#     'part_1',
#     'part_2',
#     'part_3',
#     'part_4',
#     'part_5',
#     'part_6',
#     'part_7',
#     'type_of_concept',
#     'type_of_intention',
#     'type_of_solving_question',
#     'type_of_starter'
]
categorical_columns= [
#   'user_id',
  #  'content_id',
  # 'task_container_id',
    #'part',        
    #'tags1',
   # 'bundle_id',
   # 'part_bundle_id',
    'prior_question_had_explanation',
#     'part_1',
#     'part_2',
#     'part_3',
#     'part_4',
#     'part_5',
#     'part_6',
#     'part_7',
#     'type_of_concept',
#     'type_of_intention',
#     'type_of_solving_question',
#     'type_of_starter'
]
target="answered_correctly"
flag_lgbm=True
clfs = list()
params = {
'num_leaves': 350,
'max_bin':700,
'min_child_weight': 0.03,
'feature_fraction': 0.58,
'bagging_fraction': 0.58,
#'min_data_in_leaf': 106,
'objective': 'binary',
'max_depth': -1,
'learning_rate': 0.05,
"boosting_type": "gbdt",
"bagging_seed": 11,
"metric": 'auc',
"verbosity": -1,
'reg_alpha': 0.4,
'reg_lambda': 0.6,
'random_state': 50
}
trains=list()
valids=list()
num=5
for i in range(0,num):
  
    train_df_clf=train_question.sample(n=200*100)
    print('sample end')
    #train_df.drop(train_df_clf.index, inplace=True)
    #print('train_df drop end')
    
   
    #del train_question
    
    users=train_df_clf['user_id'].drop_duplicates()
    users=users.sample(frac=0.025)
    users_df=pd.DataFrame()
    users_df['user_id']=users.values
  
  
    valid_df_newuser = pd.merge(train_df_clf, users_df, on=['user_id'], how='inner',right_index=True)
    del users_df
    del users
    gc.collect()
    #
    train_df_clf.drop(valid_df_newuser.index, inplace=True)
   

    valid_df=train_df_clf.sample(frac=0.09)
    train_df_clf.drop(valid_df.index, inplace=True)
   
    valid_df = valid_df.append(valid_df_newuser)
    del valid_df_newuser
    gc.collect()
    #

    trains.append(train_df_clf)
    valids.append(valid_df)
    print('valid_df length：',len(valid_df))
#del train_df
del train_df_clf
del valid_df
gc.collect()
for i in range(0,num):

#     
    tr_data = lgb.Dataset(trains[i][features], label=trains[i][target])
    va_data = lgb.Dataset(valids[i][features], label=valids[i][target])
    
#     del train_df_clf
#     del valid_df
#     gc.collect()
    #del trains
    #del valids
    gc.collect()

    model = lgb.train(
        params, 
        tr_data,
#         train_df[features],
#         train_df[target],
        num_boost_round=5000,
        valid_sets=[tr_data, va_data],
        early_stopping_rounds=50,
        feature_name=features,
        categorical_feature=categorical_columns,
        verbose_eval=50
    )
    clfs.append(model)
    #print('auc:', roc_auc_score(tr_data[target], model.predict(va_data[features])))
    #model.save_model(f'model.txt')
    lgb.plot_importance(model, importance_type='gain')
    plt.show()

    del tr_data
    del va_data
    gc.collect()
#    
# del trains
# del valids
# gc.collect()

In [ ]:
##setting before sakt(device+initialization seed)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # if IS_TPU == False else xm.xla_device()
#print('Running on device: {}'.format(DEVICE))
import seaborn as sns
sns.set()
DEFAULT_FIG_WIDTH = 20
sns.set_context("paper", font_scale=1.2) 
def seed_everything(s):
    random.seed(s)
    os.environ['PYTHONHASHSEED'] = str(s)
    np.random.seed(s)
    # Torch
    torch.manual_seed(s)
    torch.cuda.manual_seed(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(s)

seed = 2020
seed_everything(seed)
HOME =  "./"
DATA_HOME = "/kaggle/input/riiid-test-answer-prediction/"
MODEL_NAME = "SAKT-version_f"
MODEL_PATH = HOME + MODEL_NAME
STAGE = "stage1"
MODEL_BEST = 'model_best.pt'
FOLD = 1

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
CONTENT_TYPE_ID = "content_type_id"
CONTENT_ID = "content_id"
TARGET = "answered_correctly"
USER_ID = "user_id"
TASK_CONTAINER_ID = "task_container_id"
TIMESTAMP = "timestamp" 
dtype = {TIMESTAMP: 'int64', USER_ID: 'int32', CONTENT_ID: 'int16',CONTENT_TYPE_ID: 'bool',TARGET:'int8'}
train_df = pd.read_csv(DATA_HOME + 'train.csv', usecols=[1, 2, 3, 4, 7], dtype=dtype,nrows=15000000)
#train_df = train.iloc[:,[1, 2, 3, 4, 7]]
train_df = train_df[train_df[CONTENT_TYPE_ID] == False].reset_index(drop = True)
train_df.head()
# Valid
valid_df = train_df.groupby([USER_ID]).tail(100)
print("valid:", valid_df.shape, "users:", valid_df[USER_ID].nunique())
valid_df = valid_df.reset_index(drop = True)
valid_group = valid_df[[USER_ID, CONTENT_ID, TARGET]].groupby(USER_ID).apply(lambda r: (r[CONTENT_ID].values, r[TARGET].values))
# Train
train_df.drop(valid_df.index, inplace = True)
train_df = train_df.reset_index(drop = True)
train_group = train_df[[USER_ID, CONTENT_ID, TARGET]].groupby(USER_ID).apply(lambda r: (r[CONTENT_ID].values, r[TARGET].values))
print("train:", train_df.shape, "users:", train_df[USER_ID].nunique())
skills = train_df[CONTENT_ID].unique()
n_skill =13523
del train_df
del valid_df

In [ ]:
class conf:
    METRIC_ = "max"
    WORKERS = 4 # 0
    BATCH_SIZE = 2048
    lr = 1e-3
    D_MODEL = 128

    if torch.cuda.is_available():
        map_location=lambda storage, loc: storage.cuda()
    else:
        map_location='cpu'

In [ ]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=100, embed_dim=128):
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)

        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        device = x.device        
        x = self.embedding(x)
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)

        pos_x = self.pos_embedding(pos_id)
        x = x + pos_x

        e = self.e_embedding(question_ids)

        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1), att_weight

In [ ]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_skill, subset="train", max_seq=100):
        super(SAKTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill 
        self.samples = group
        self.subset = subset
        
        # self.user_ids = [x for x in group.index]
        self.user_ids = []
        for user_id in group.index:
            q, qa = group[user_id]
            if len(q) < 10: # 10 interactions minimum
                continue
            self.user_ids.append(user_id) # user_ids indexes

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index] # Pick a user
        q_, qa_ = self.samples[user_id] # Pick full sequence for user
        seq_len = len(q_)

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)

        if seq_len >= self.max_seq:
            if self.subset == "train":
                if seq_len > self.max_seq:
                    random_start_index = np.random.randint(seq_len - self.max_seq)
                    q[:] = q_[random_start_index:random_start_index + self.max_seq] # Pick 100 questions from a random index
                    qa[:] = qa_[random_start_index:random_start_index + self.max_seq] # Pick 100 answers from a random index
                else:
                    q[:] = q_[-self.max_seq:]
                    qa[:] = qa_[-self.max_seq:]
            else:
                q[:] = q_[-self.max_seq:] # Pick last 100 questions
                qa[:] = qa_[-self.max_seq:] # Pick last 100 answers
        else:
            q[-seq_len:] = q_ # Pick last N question with zero padding
            qa[-seq_len:] = qa_ # Pick last N answers with zero padding        
                
        target_id = q[1:] # Ignore first item 1 to 99
        label = qa[1:] # Ignore first item 1 to 99

        # x = np.zeros(self.max_seq-1, dtype=int)
        x = q[:-1].copy() # 0 to 98
        x += (qa[:-1] == 1) * self.n_skill # y = et + rt x E

        return x, target_id, label

In [ ]:
def train_epoch(model_sakt, train_iterator, optim, criterion, device="cpu"):
    model_sakt.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    # tbar = tqdm(train_iterator)
    for item in train_iterator: #tbar:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()

        optim.zero_grad()
        output, atten_weight = model_sakt(x, target_id)
        loss = criterion(output, label)
        loss.backward()
        optim.step()
        train_loss.append(loss.item())

        output = output[:, -1]
        label = label[:, -1] 
        pred = (torch.sigmoid(output) >= 0.5).long()
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        #outs.extend(output.view(-1).data.cpu().numpy())
        outs.extend(torch.sigmoid(output).view(-1).data.cpu().numpy())

        # tbar.set_description('loss - {:.4f}'.format(loss))
    
    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc
def valid_epoch(model_sakt, valid_iterator, criterion, device="cpu"):
    model_sakt.eval()

    valid_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []

    #tbar = tqdm(valid_iterator)
    for item in valid_iterator: # tbar:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()

        with torch.no_grad():
            output, atten_weight = model_sakt(x, target_id)
        loss = criterion(output, label)
        valid_loss.append(loss.item())

        output = output[:, -1] # (BS, 1)
        label = label[:, -1] 
        pred = (torch.sigmoid(output) >= 0.5).long()
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        #outs.extend(output.view(-1).data.cpu().numpy())
        outs.extend(torch.sigmoid(output).view(-1).data.cpu().numpy())

        #tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(valid_loss)

    return loss, acc, auc

In [ ]:
train_dataset = SAKTDataset(train_group, n_skill, subset="train")
train_dataloader = DataLoader(train_dataset, batch_size=conf.BATCH_SIZE, shuffle=True, num_workers=conf.WORKERS)

valid_dataset = SAKTDataset(valid_group, n_skill, subset="valid")
valid_dataloader = DataLoader(valid_dataset, batch_size=conf.BATCH_SIZE, shuffle=False, num_workers=conf.WORKERS)
item = train_dataset.__getitem__(5)
#print("x", len(item[0]), item[0])
#print("target_id", len(item[1]), item[1])
#print("label", len(item[2]), item[2])

In [ ]:
device = DEVICE

model_sakt = SAKTModel(n_skill, embed_dim=conf.D_MODEL)
optimizer = torch.optim.Adam(model_sakt.parameters(), lr=conf.lr)
criterion = nn.BCEWithLogitsLoss()

model_sakt.to(device)
criterion.to(device)

In [ ]:
epochs = 20
auc_max = -np.inf
history = []

snapshot_path = "%s/fold%d/%s/snapshots" % (MODEL_PATH, FOLD, STAGE)
if not os.path.exists(snapshot_path):
    os.makedirs(snapshot_path)

print("Stage:", STAGE, "fold:", FOLD, "on:", DEVICE, "workers:", conf.WORKERS, "batch size:", conf.BATCH_SIZE, "metric_:", conf.METRIC_, 
      "train dataset:", len(train_dataset), "valid dataset:", len(valid_dataset))

for epoch in range(1, epochs+1):
    train_loss, train_acc, train_auc = train_epoch(model_sakt, train_dataloader, optimizer, criterion, device)
    print("\nEpoch#{}, train_loss - {:.2f} acc - {:.4f} auc - {:.4f}".format(epoch, train_loss, train_acc, train_auc))
    valid_loss, valid_acc, valid_auc = valid_epoch(model_sakt, valid_dataloader, criterion, device)
    print("Epoch#{}, valid_loss - {:.2f} acc - {:.4f} auc - {:.4f}".format(epoch, valid_loss, valid_acc, valid_auc))
    lr = optimizer.param_groups[0]['lr']
    history.append({"epoch":epoch, "lr": lr, **{"train_auc": train_auc, "train_acc": train_acc}, **{"valid_auc": valid_auc, "valid_acc": valid_acc}})
    if valid_auc > auc_max:
        print("Epoch#%s, valid loss %.4f, Metric loss improved from %.4f to %.4f, saving model ..." % (epoch, valid_loss, auc_max, valid_auc))
        auc_max = valid_auc
        torch.save(model_sakt.state_dict(), os.path.join(snapshot_path, MODEL_BEST))

if history:
    metric = "auc"
    # Plot training history
    history_pd = pd.DataFrame(history[1:]).set_index("epoch")
    train_history_pd = history_pd[[c for c in history_pd.columns if "train_" in c]]
    valid_history_pd = history_pd[[c for c in history_pd.columns if "valid_" in c]]
    lr_history_pd = history_pd[[c for c in history_pd.columns if "lr" in c]]
    fig, ax = plt.subplots(1,2, figsize=(DEFAULT_FIG_WIDTH, 6))
    t_epoch = train_history_pd["train_%s" % metric].argmin() if conf.METRIC_ == "min" else train_history_pd["train_%s" % metric].argmax()
    v_epoch = valid_history_pd["valid_%s" % metric].argmin() if conf.METRIC_ == "min" else valid_history_pd["valid_%s" % metric].argmax()
    d = train_history_pd.plot(kind="line", ax=ax[0], title="Epoch: %d, Train: %.3f" % (t_epoch, train_history_pd.iloc[t_epoch,:]["train_%s" % metric]))
    d = lr_history_pd.plot(kind="line", ax=ax[0], secondary_y=True)
    d = valid_history_pd.plot(kind="line", ax=ax[1], title="Epoch: %d, Valid: %.3f" % (v_epoch, valid_history_pd.iloc[v_epoch,:]["valid_%s" % metric]))
    d = lr_history_pd.plot(kind="line", ax=ax[1], secondary_y=True)
    plt.savefig("%s/train.png" % snapshot_path, bbox_inches='tight')
    plt.show()